In [1]:
# читаем датасет
import numpy as np
import pandas as pd
# load dataset from file csv
# f = 'D:/koltsov_python_scripts/Tomotopy/preprocessing_data/20news_lem.csv'
f = 'C:/koltcov_python_scripts/Word_embeding project/coherence/lenta_lem_2_utf8.csv'
#f = 'D:/koltsov_python_scripts/Tomotopy/preprocessing_data/WoS11967_lem_1.csv'

df=pd.read_csv(f,  sep=';', encoding='utf8')
#df=pd.read_csv(f,  sep=';', encoding='cp1252' )
#df=pd.read_csv(f,  sep=';', encoding='Latin-1')
# df=pd.read_csv(f,  sep=';', encoding='ANSI')

# this gives us the size of the array
print('Dataset size', df.shape)

# here we can get size of array as two variables
num_rows, num_feature = df.shape

print('row number: ', num_rows)
print('feature number: ', num_feature)
print()
print('names of features: ', list(df))

print('-------------------')
print('full data loaded')
print('-------------------')

Dataset size (8650, 1)
row number:  8650
feature number:  1

names of features:  ['doc']
-------------------
full data loaded
-------------------


In [2]:
import nltk
# download Tokenizer 
nltk.download("punkt")
# создание объекта nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

# here we delited list
y_data = []

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import re
import time
# для того что бы огранизовать лематизацию пр помощи pymorphy2 организуем цикл по текстам
# there will be a list of original texts
x_data = []
# there will be a list of lematized texts
y_data = []

start = time.time()

for i in range (num_rows):
    doc_stem =''
    s1 = df['doc'][i]

    # удаляем не нужные символы
    s2 =s1.replace(',', '')
    s3 =s2.replace('.', '')
    s4 = re.sub('[!@#$1234567890#—ツ►๑۩۞۩•*”˜˜”*°°*`]', '', s3)
    tokens = word_tokenize(s3)
    
    for j in range(0,len(tokens)):
        tokens[j] = morph.parse(tokens[j])[0].normal_form
        doc_stem = doc_stem + tokens[j] + ' '
    
    #print(doc_stem)
    
    #print('-----------------------------------------------------------------------------')
    #x_data.append(df['doc'][i])
    y_data.append(doc_stem)

stop = time.time()    
# results is set lematized documents
print('time of execution (sec)', stop - start)

time of execution (sec) 91.71959733963013


In [4]:
# The CountVectorizer module in sklearn allows you to convert a set of texts into a matrix of tokens in the text. 
from sklearn.feature_extraction.text import CountVectorizer
# Convert a collection of text documents to a matrix of token counts
vec = CountVectorizer()

x = vec.fit_transform(y_data)
data = vec.fit_transform(y_data).toarray()

num_docs, num_feature =x.shape
print('number of documents: ', num_docs,'----', 'number of features: ', num_feature)

number of documents:  8650 ---- number of features:  11222


In [42]:
print(len(vec.vocabulary_))
print(len(vec.get_feature_names()))

11222
11222


In [43]:
import gensim
from gensim import corpora
start = time.time()
# Tokenize(split) the sentences into words
texts = [[text for text in doc.split()] for doc in y_data]
# Create dictionary
dictionary = corpora.Dictionary(texts)
# create a corpus
gensim_dictionary = corpora.Dictionary(texts)
gensim_corpus = [gensim_dictionary.doc2bow(token, allow_update=True) for token in texts]

# Get information about the dictionary and corpus
print(dictionary)
print('number of tokens: ', len(dictionary))
print('number of docs: ', len(texts))
print('number of docs: ', len(gensim_corpus))
stop = time.time()    
# results is set lematized documents
print('time of execution (sec)', stop - start)

Dictionary(11226 unique tokens: ['аляска', 'анонсировать', 'баллистический', 'безопасность', 'беспрецедентный']...)
number of tokens:  11226
number of docs:  8650
number of docs:  8650
time of execution (sec) 3.043997049331665


итак, мы подготовили документы и словарь для расчета coherence
теперь нужно считать матрицу рапределения слов и из нее вытажить 
списки топовых слов по каждой теме.

In [108]:
# читаем матрицу слов как датафрейм
f = 'C:/koltcov_python_scripts/Word_embeding project/coherence/rus_vectors_embeddings_19_train_1_200_Phi.csv'
phidf=pd.read_csv(f,  sep=';', encoding='ANSI')
# this gives us the size of the array
print('Dataset size', phidf.shape)

# here we can get size of array as two variables
num_rows, num_feature = phidf.shape

print('row number: ', num_rows)
print('feature number: ', num_feature)
print()
print('names of features: ', list(phidf))

print('-------------------')
print('full data loaded')
print('-------------------')

Dataset size (7867, 20)
row number:  7867
feature number:  20

names of features:  ['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']
-------------------
full data loaded
-------------------


In [110]:
phidf=phidf.rename(columns={'Unnamed: 0': 'words'})
phidf.head(10)

,words,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,корпорация,8.902018e-06,1.654974e-06,1.458763e-03,4.119167e-04,3.746196e-06,2.206045e-06,1.907293e-06,9.280930e-07,0.000001,1.617924e-03,4.180690e-05,4.492129e-06,5.368213e-06,2.675426e-06,6.017845e-06,0.000133,0.000006,4.505778e-07,2.250381e-06
1,универмаг,8.202667e-07,9.632521e-07,2.827780e-06,7.055913e-07,3.106353e-07,1.475976e-06,4.086891e-07,1.044617e-06,0.000004,9.965071e-07,1.448219e-06,3.187734e-07,2.062774e-06,8.050685e-07,3.282399e-07,0.000312,0.000091,1.678579e-06,6.879072e-07
2,louis,9.143426e-07,5.843416e-07,2.653134e-06,1.637984e-06,2.126544e-07,1.419391e-06,7.456376e-07,1.025717e-06,0.000002,1.337204e-06,6.334690e-07,2.250528e-07,1.749954e-06,1.556454e-06,2.241893e-07,0.000208,0.000003,1.138570e-06,1.096217e-06
3,фетис,4.352915e-07,9.881717e-07,2.196915e-05,1.982393e-06,7.669111e-07,1.164320e-05,1.836156e-06,1.686463e-04,0.000004,1.478178e-06,2.361681e-06,8.145795e-07,4.386285e-06,3.620375e-06,7.432017e-07,0.000004,0.000002,2.606110e-06,1.536725e-06
4,возбуждение,1.730360e-06,5.729775e-04,4.982857e-07,7.165572e-07,1.212593e-06,5.225035e-07,2.193131e-06,1.095204e-06,0.000003,1.208826e-06,8.873915e-06,1.776835e-06,7.891902e-05,2.358903e-06,9.682977e-07,0.000001,0.000008,9.839202e-06,1.281997e-03
5,джонатан,1.348272e-06,1.822061e-06,1.503479e-06,7.644163e-07,1.028842e-06,1.848728e-06,8.165979e-07,3.445128e-06,0.000030,1.915792e-06,7.057784e-07,1.011118e-06,6.031982e-05,2.606348e-06,1.060634e-06,0.000296,0.000007,2.130533e-06,8.720277e-07
6,моника,1.800773e-06,2.901000e-05,7.516395e-07,5.516647e-07,1.311752e-06,1.643767e-04,2.318481e-06,2.235558e-06,0.000038,2.676147e-06,7.383351e-07,1.172423e-06,9.042326e-07,3.592328e-06,9.039379e-07,0.000157,0.000005,2.144181e-05,4.224069e-06
7,кристофер,2.093107e-06,1.763958e-05,2.713066e-06,1.698303e-05,4.332009e-06,1.939515e-06,1.049077e-06,1.451365e-05,0.000038,1.736215e-06,1.909535e-06,4.350151e-06,2.575525e-05,3.284546e-06,4.116194e-06,0.000154,0.000074,1.592024e-05,1.813317e-06
8,занятость,2.592312e-06,2.468589e-06,2.285690e-06,1.580755e-04,5.496201e-06,4.589315e-06,3.333927e-06,1.439194e-06,0.000011,8.224458e-07,6.927769e-06,6.352322e-06,1.785996e-06,3.631725e-06,4.600655e-06,0.000002,0.000005,6.866523e-05,3.331691e-04
9,осматривать,7.817801e-06,1.589077e-04,3.861429e-06,3.376359e-07,3.201790e-06,2.276997e-06,8.961468e-07,1.283730e-06,0.000203,6.684770e-05,1.769789e-06,3.440636e-06,3.579619e-06,1.428577e-06,3.225548e-06,0.000005,0.000004,3.212048e-06,2.198661e-06


теперерь нужно отсортиоовать по вероятности в каждой колонке, 
что бы получить наиболее вероятностеые слова,
то есть содержимое каждой темы

In [113]:
sorted_df = phidf.sort_values(by='0', ascending = False)
print(sorted_df['words'].head(10).to_list())

['организация', 'террористический', 'россия', 'запрещать', 'группировка', 'сирия', 'государство', 'исламский', 'территория', 'боевик']


In [114]:
# организуем сортировку по каждой колонке, 
# и будем сохранять слова в колонке 'words'
mytopics = []
# создаем список мен столбцов
topics_name =list(phidf)
topics_name.remove('words')
# цикл по именам столбцов
for i in topics_name:
    # сортируем по текущему столбцу
    sorted_df = phidf.sort_values(by=i, ascending = False)
    mytopics.append(sorted_df['words'].head(5).to_list())

In [115]:
print(mytopics)

[['организация', 'террористический', 'россия', 'запрещать', 'группировка'], ['полиция', 'происходить', 'мужчина', 'летний', 'сотрудник'], ['компания', 'российский', 'процент', 'тысяча', 'миллиард'], ['доллар', 'рубль', 'россия', 'процент', 'банк'], ['проводить', 'тысяча', 'работа', 'страна', 'место'], ['россия', 'президент', 'российский', 'украина', 'владимир'], ['россия', 'страна', 'отношение', 'министр', 'санкция'], ['россия', 'чемпионат', 'команда', 'российский', 'матч'], ['ребёнок', 'город', 'летний', 'штат', 'женщина'], ['самолёт', 'ракета', 'военный', 'борт', 'километр'], ['россия', 'закон', 'правительство', 'президент', 'глава'], ['работа', 'проводить', 'тысяча', 'страна', 'место'], ['учёный', 'исследование', 'журнал', 'результат', 'обнаруживать'], ['президент', 'партия', 'выбор', 'кандидат', 'пост'], ['проводить', 'работа', 'тысяча', 'страна', 'место'], ['компания', 'фильм', 'основывать', 'выпускать', 'представлять'], ['сеть', 'пользователь', 'видео', 'соцсеть', 'страница'], ['

In [118]:
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(topics=mytopics, corpus=gensim_corpus, dictionary=gensim_dictionary, coherence='u_mass')
coherence = cm.get_coherence()  # get coherence value
print(coherence)

-1.6245082112055202
